In [74]:
import pickle
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import KNNImputer

from sklearn.pipeline import Pipeline
import numpy as np

# Preprocessing

## Biomed

In [43]:
with open("data/biomed/preprocessed_biomed_data.pickle", "rb") as f:
    biomed_x = pickle.load(f)
biomed_x

,Age_of_patient,ml,m2,m3,m4
141,33,31.0,75.5,13.7,160.0
144,30,34.0,75.0,15.4,171.0
37,42,78.0,118.0,15.5,212.0
124,22,22.0,85.5,15.0,NaN
23,32,485.0,83.5,73.0,382.0
...,...,...,...,...,...
60,52,197.0,91.5,25.2,236.0
107,25,35.0,71.0,8.8,186.0
18,30,40.0,85.5,12.7,201.0
63,39,154.0,103.5,21.3,296.0


In [44]:
with open("data/biomed/y_biomed.pickle", "rb") as f:
    biomed_y = pickle.load(f)
biomed_y

0      carrier
1      carrier
2      carrier
3      carrier
4      carrier
        ...   
204     normal
205     normal
206     normal
207     normal
208     normal
Name: class, Length: 209, dtype: category
Categories (2, object): ['carrier' < 'normal']

## Fertility


In [45]:
with open("data/fertility/preprocessed_fertility_data.pickle", "rb") as f:
    fertility_x = pickle.load(f)
fertility_x

,age,child_diseases,accident,surgery,hours_sitting,fall,spring,summer,winter,fever_never,fever_not_recent,fever_recent,smoking_daily,smoking_never,smoking_occasionally,alcohol_daily,alcohol_rarely_or_never,alcohol_several_daily,alcohol_several_weekly,alcohol_weekly
35,0.78,0,0,1,0.38,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0
13,0.81,0,1,1,0.38,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0
21,0.75,0,1,1,0.25,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0
26,0.67,0,1,0,0.38,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0
50,0.67,0,1,0,0.19,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,1.00,0,0,0,0.38,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0
10,0.67,0,0,1,0.31,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1
45,0.53,0,1,1,0.44,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0
17,0.69,0,1,0,0.25,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1


In [46]:
with open("data/fertility/y_fert.pickle", "rb") as f:
    fertility_y = pickle.load(f)
fertility_y

0     0
1     1
2     0
3     0
4     1
     ..
95    0
96    0
97    0
98    0
99    0
Name: fertility, Length: 100, dtype: category
Categories (2, int64): [0 < 1]

## Wine

https://www.kaggle.com/datasets/shelvigarg/wine-quality-dataset

Abstract: Datasets included, related to red and white Vinho Verde wine samples, from the north of Portugal. The goal is to model wine quality based on physicochemical tests (see [Cortez et al., 2009], [Web Link: http://www3.dsi.uminho.pt/pcortez/wine/]).

Data Set Characteristics: Multivariate

Number of Instances:4898

Number of Attributes: 12

Missing Values: N/A

Source: 
Paulo Cortez, University of Minho, Guimarães, Portugal, http://www3.dsi.uminho.pt/pcortez 
A. Cerdeira, F. Almeida, T. Matos and J. Reis, Viticulture Commission of the Vinho Verde Region(CVRVV), Porto, Portugal


Input variables (based on physicochemical tests):
- fixed acidity
- volatile acidity
- citric acid
- residual sugar
- chlorides
- free sulfur dioxide
- total sulfur dioxide
- density
- pH
- sulfates
- alcohol
Output variable (based on sensory data): 
quality (score between 0 and 10)




winequality.csv

In [47]:
with open("data/wine/winequality.csv", "rb") as f:
    wine = pd.read_csv(f)
wine

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,white,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,white,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,white,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,red,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
6493,red,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,NaN,11.2,6
6494,red,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
6495,red,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [48]:
# NAs: drop
wine = wine.dropna()
wine.shape

(6463, 13)

In [49]:
# type: 1-hot encoded
wine["white"] = wine.type.apply(lambda x: 1 if x == "white" else 0)
wine["red"] = wine.type.apply(lambda x: 1 if x == "red" else 0)
wine.drop("type", axis=1, inplace=True)
wine

/var/folders/wn/h7d3tkjj12g2j3lhccrbhnlc0000gn/T/ipykernel_46086/1311090816.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wine["white"] = wine.type.apply(lambda x: 1 if x == "white" else 0)
/var/folders/wn/h7d3tkjj12g2j3lhccrbhnlc0000gn/T/ipykernel_46086/1311090816.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wine["red"] = wine.type.apply(lambda x: 1 if x == "red" else 0)
/var/folders/wn/h7d3tkjj12g2j3lhccrbhnlc0000gn/T/ipykernel_46086/1311090816.py:4: SettingWithCopyWarning: 
A value is trying

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,white,red
0,7.0,0.270,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6,1,0
1,6.3,0.300,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6,1,0
2,8.1,0.280,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6,1,0
3,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,1,0
4,7.2,0.230,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6491,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6,0,1
6492,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,0,1
6494,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,0,1
6495,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,0,1


In [50]:
# separate x and y data
wine_x = wine
wine_y = wine_x.pop("quality")

In [51]:
# scale the data
preprocessor = Pipeline(
   steps=[
       ("scaler", MinMaxScaler())
    ]
)
cols = wine_x.columns
wine_x = pd.DataFrame(preprocessor.fit_transform(wine_x))
wine_x.columns = cols
wine_x


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,white,red
0,0.264463,0.126667,0.216867,0.308282,0.059801,0.152778,0.377880,0.267785,0.217054,0.129213,0.115942,1.0,0.0
1,0.206612,0.146667,0.204819,0.015337,0.066445,0.045139,0.290323,0.132832,0.449612,0.151685,0.217391,1.0,0.0
2,0.355372,0.133333,0.240964,0.096626,0.068106,0.100694,0.209677,0.154039,0.418605,0.123596,0.304348,1.0,0.0
3,0.280992,0.100000,0.192771,0.121166,0.081395,0.159722,0.414747,0.163678,0.364341,0.101124,0.275362,1.0,0.0
4,0.280992,0.100000,0.192771,0.121166,0.081395,0.159722,0.414747,0.163678,0.364341,0.101124,0.275362,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6458,0.247934,0.360000,0.048193,0.019939,0.098007,0.093750,0.073733,0.181222,0.542636,0.337079,0.217391,0.0,1.0
6459,0.198347,0.346667,0.048193,0.021472,0.134551,0.107639,0.087558,0.150183,0.565891,0.202247,0.362319,0.0,1.0
6460,0.206612,0.286667,0.078313,0.026074,0.111296,0.097222,0.078341,0.166377,0.542636,0.297753,0.434783,0.0,1.0
6461,0.173554,0.376667,0.072289,0.021472,0.109635,0.107639,0.087558,0.161172,0.658915,0.275281,0.318841,0.0,1.0


## Congress

In [75]:
with open("data/congress/CongressionalVotingID.shuf.lrn.csv", "rb") as f:
    congress_x = pd.read_csv(f)
# dorp unnecessary axis
congress_x = congress_x.drop("ID", axis=1)
# replace "unknown" with proper na
congress_x.replace('unknown', np.nan, inplace=True)
# replace y and n with 1 and 0
congress_x.replace({'y': 1, 'n': 0}, inplace=True)
# remove the labels as a separate pd.series
congress_y = congress_x.pop("class")
# impute missing values in features
preprocessor = Pipeline(
   steps=[
       ("imputation", KNNImputer(n_neighbors=2, weights="uniform")),
       ("scaler", MinMaxScaler())
    ]
)
cols = congress_x.columns
congress_x = pd.DataFrame(preprocessor.fit_transform(congress_x))
congress_x.columns = cols

congress_x

,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-crporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,0.478673,0.497462,0.61215,0.386792,0.473684,0.615023,0.59434,0.615385,0.509709,0.495327,0.346341,0.381188,0.495098,0.530806,0.470588,0.8125
1,1.000000,0.000000,1.00000,0.000000,0.000000,1.000000,1.00000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0000
2,1.000000,1.000000,0.00000,1.000000,1.000000,1.000000,0.00000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.5000
3,0.000000,0.000000,1.00000,0.000000,0.000000,0.000000,1.00000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.0000
4,0.000000,0.000000,1.00000,1.000000,1.000000,1.000000,0.00000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,1.000000,1.000000,1.00000,0.000000,0.000000,0.000000,0.00000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.0000
214,1.000000,1.000000,1.00000,0.000000,0.000000,0.000000,1.00000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0000
215,1.000000,0.000000,0.00000,1.000000,1.000000,0.000000,1.00000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.0000
216,0.000000,0.000000,1.00000,1.000000,1.000000,1.000000,1.00000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0000


In [76]:
congress_y

0      republican
1        democrat
2        democrat
3        democrat
4      republican
          ...    
213      democrat
214      democrat
215    republican
216    republican
217      democrat
Name: class, Length: 218, dtype: object